# Westcott $g$ factors

This `notebook` can be used to demonstrate methods available in the `westcott` Python library.

In [ ]:
import westcott
gw = westcott.Westcott()
import numpy as np

In [ ]:
# Find list of available targets
t=gw.find_targets()
print(*t)

## Maxwellian distributions

In [ ]:
# Get ENDF-B/VIII.1 cross section data
endf_e, endf_cs = gw.sigma_ENDF('Sm149')

In [ ]:
# Evaluate g-factor based on Maxwellian distribution for various temperatures
T = [100,200,293,400,500,600]
print("Maxwellian g-factors")
for t in T:
    gW = gw.gw_Maxwellian(t, endf_e, endf_cs)
    print(f"T = {t} K: g = {gW}")

In [ ]:
# Repeat for 83Kr
endf_e, endf_cs = gw.sigma_ENDF('Kr83')
T = [100,200,293,400,500,600]
print("Maxwellian g-factors")
for t in T:
    gW = gw.gw_Maxwellian(t, endf_e, endf_cs)
    print(f"T = {t} K: g = {gW}")

## Experimental neutron flux distributions

In [ ]:
# Find list of tuples corresponding to the available experimental spectra
gw.find_flux()

In [ ]:
# Get flux data for cold FRM-II spectrum
energy, cs = gw.get_flux(0)

In [ ]:
# Evaluate g-factor based on cold FRM-II flux
endf_e, endf_cs = gw.sigma_ENDF('Sm149')
gW = gw.gw_arbitrary(energy, cs, endf_e, endf_cs)
print(f"FRM-II cold-neutron spectrum (21 K) g = {gW}")

In [ ]:
# Repeat for 83Kr 
endf_e, endf_cs = gw.sigma_ENDF('Kr83')
gW = gw.gw_arbitrary(energy, cs, endf_e, endf_cs)
print(f"FRM-II cold-neutron spectrum (21 K) g = {gW}")

In [ ]:
# Get flux data for thermal BRR spectrum
energy, cs = gw.get_flux(1)

In [ ]:
# Evaluate g-factor based on thermal BRR flux
endf_e, endf_cs = gw.sigma_ENDF('Sm149')
gW = gw.gw_arbitrary(energy, cs, endf_e, endf_cs)
print(f"BRR thermal-neutron spectrum (293 K) g = {gW}")

In [ ]:
# Repeat for 83Kr 
endf_e, endf_cs = gw.sigma_ENDF('Kr83')
gW = gw.gw_arbitrary(energy, cs, endf_e, endf_cs)
print(f"BRR thermal-neutron spectrum (293 K) g = {gW}")

## Resonance parameters from ENDF

In [ ]:
# Find lists of targets with different types of resonance parametrizations
targs = gw.find_resonances()
print("All targets with resonance parameters:\n")
print(*targs)
bw = gw.find_resonances(res='BW')
print("\nTargets with Breit-Wigner resonance parameters:\n")
print(*bw)
rm = gw.find_resonances(res='RM')
print("\nTargets with Reich-Moore resonance parameters:\n")
print(*rm)

### Irregularity: Breit-Wigner

In [ ]:
# Get resonance parameters for Breit-Wigner 83Kr
target_bw = 'Kr83'
res_bw = gw.get_res_paras(target_bw)
res_bw

In [ ]:
# Extract first resonance needed for irregularity approximation
res_array = res_bw.to_numpy()
res_e = res_array[0][0]
res_width_tot = res_array[0][3]
print(f"Energy of first resonance = {res_e} eV")
print(f"Total width first resonance = {res_width_tot}")

In [ ]:
# Calculate Westcott g-factor based on irregularity approximation for various temperatures
T = [100,200,293,400,500,600]
print("Irregularity g-factors")
for t in T:
    gW = gw.gw_irregularity(res_e, res_width_tot, t)
    print(f"T = {t} K: g = {gW}")

In [ ]:
# Create simple bar plot of resonances from DataFrame
res_bw.plot(x='energy', y='totalWidth', kind='bar')

In [ ]:
# Dump DataFrame to CSV
res_bw.to_csv("resonances_bw_{0}.csv".format(target_bw), index=False)

### Irregularity: Reich-Moore

In [ ]:
# Get resonance parameters for Reich-Moore 157Gd
target_rm = 'Gd157'
res_rm = gw.get_res_paras(target_rm)
res_rm

In [ ]:
# Extract first resonance needed for irregularity approximation
res_array = res_rm.to_numpy()
res_e = res_array[0][0]
res_width_tot = res_array[0][1]+res_array[0][2]
print(f"Energy of first resonance = {res_e} eV")
print(f"Total width first resonance = {res_width_tot}")

In [ ]:
# Calculate Westcott g-factor based on irregularity approximation for various temperatures
T = [100,200,293,400,500,600]
print("Irregularity g-factors")
for t in T:
    gW = gw.gw_irregularity(res_e, res_width_tot, t)
    print(f"T = {t} K: g = {gW}")

In [ ]:
# Use regex methods to determine 'A+1' compound system
import re
letters_pattern = '\D+'
numbers_pattern = '\d+'
chem_symbol = str(re.findall(letters_pattern, target_rm)[0])
mass = int(re.findall(numbers_pattern, target_rm)[0])
compound_mass = mass + 1
compound = str(chem_symbol+str(compound_mass))
print(f"{target_rm} + n -> {compound}")

In [ ]:
# Combine DataFrame columns to calculate total witdth then create simple scatter plot of resonances from DataFrame
res_rm['totalWidth'] = res_rm[f'{compound} + photon [inclusive] width'] + res_rm[f'n + {target_rm} width']
res_rm.plot(x='energy',y='totalWidth',kind='scatter')

In [ ]:
# Dump DataFrame to CSV with 'totalWidth' in final column
res_rm.to_csv("resonances_rm_{0}.csv".format(target_rm), index=False)

## Inspect ENDF neutron-capture cross-section data

In [ ]:
import matplotlib.pyplot as plt
import re

In [ ]:
def plot_MT102(df,target,energy_units='eV',cs_units='b',save=False):
    """Function to plot the ENDF neutron-capture cross-section data corresponding to MT=102."""    
    letters_pattern = '\D+'
    numbers_pattern = '\d+'
    chem_symbol = str(re.findall(letters_pattern, target)[0])
    mass = int(re.findall(numbers_pattern, target)[0])
    regex_label = r'$^{%i}$%s($n,\gamma$)'%(mass, chem_symbol)
    
    energy=df['energy [eV]'].tolist()
    cs=df['cross section [b]'].tolist()
    
    if energy_units == 'MeV': energy = [x/1e+06 for x in energy]
    else: pass
    if cs_units == 'mb': cs = [x*1e+03 for x in cs]
    else: pass
    
    f, ax = plt.subplots(figsize=(8,5))
    ax.plot(energy, cs, color='black', label=regex_label)
    ax.legend(loc='best', fontsize=15)
    ax.set_xlabel(r'$E$ [{0}]'.format(energy_units),size=15)
    ax.set_ylabel(r'$\sigma_{\gamma}$ [%s]'%cs_units,size=15)
    ax.grid(True)
    ax.set_xscale('log')
    ax.set_yscale('log')

    plt.tight_layout()
    plt.show()
    if save==True:
        plt.savefig("ng_cross_section_{0}.png".format(target),dpi=f.dpi)
    else:
        pass
    return

In [ ]:
# Define ENDF target for inspection
target='Gd157'

In [ ]:
# Plot the ENDF cross-section data
df = gw.get_MT102(target)
plot_MT102(df,target,energy_units='MeV',cs_units='mb')
# To save the figure:
#plot_MT102(df,target,energy_units='MeV',cs_units='mb',save=True)

In [ ]:
# Dump ENDF data (cross section in [b]; energy in [eV]) to CSV
df.to_csv("ng_cross_section_{0}.csv".format(target), index=False)